In [23]:
import requests
import time
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
import time
import time
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

extracted_text = []
aux_pages = 0
links = pd.read_csv(r'..\src\data\raw\references\dominios_selecionados.csv',
                    sep='|',
                    usecols=['links', 'journal'])
links = links.rename(columns={'links':'link'})

In [24]:
class GetData:
    def __init__(self):
        pass

    def from_reddit(self, url: str, qtd_chamadas: int=5) -> list[dict]:
        json_result_list = []
        tentativa = qtd_chamadas
        aux = 0

        # Primeira requisição
        response = requests.get(url)
        json_object = response.json()

        next_page = json_object['data']['after']
        data_exists = json_object['data']

        for item in json_object['data']['children']:
            aux = aux+1
        for page in range(aux):
            json_result_list.append(json_object['data']['children'][page]['data']['selftext'])

        # Loop para paginação
        while data_exists:
            if tentativa > 0:
                time.sleep(20)
                print("Next page:", next_page)
                response = requests.get(url, params={'after': next_page})
                json_object = response.json()

                # Pega a chave selftext de cada post, por página
                aux = 0
                for item in json_object['data']['children']:
                    aux = aux+1
                for page in range(aux):
                    json_result_list.append(json_object['data']['children'][page]['data']['selftext'])

                next_page = json_object['data'].get('after')
                data_exists = json_object['data']
                tentativa = tentativa - 1
            else: break
        return json_result_list
          
    def from_website(self, references_links: pd.DataFrame) -> pd.DataFrame:
        options = Options()
        options.add_argument('--headless=new')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')
        options.add_argument('--log-level=3')
        extracted_text = []

        driver = webdriver.Chrome(options=options)

        for _, row in references_links.iterrows():
            journal = row['journal']
            link = row['link']
            try:
                print(f'Getting data from: {link}')
                driver.get(link)
                time.sleep(5)

                if journal == 'einvestidor':
                    xpath = '/html/body/div[2]/article/div[3]'
                elif journal == 'canaltech':
                    xpath = '/html/body/div[1]/div[2]/main/div/section/div/div[2]/div[1]/div/div[3]'
                elif journal == 'olhar':
                    xpath = '/html/body/div[4]/div/main/article'
                elif journal == 'noticias_uol':
                    xpath = '/html/body/div[1]/main/article'
                elif journal == 'em':
                    xpath = '/html/body/main/section[2]/div/div/div/div[1]/article/div[2]'
                elif journal == 'tecmundo':
                    xpath = '/html/body/div/main/div/div/main/div[5]/div/div[1]'
                elif journal == 'terra':
                    xpath = '/html/body/article/div[4]/div/div[1]/div[2]'
                elif journal == 'cnnbrasil':
                    xpath = '/html/body/div[2]/article/div[3]'
                elif journal == 'valorinveste':
                    xpath = '/html/body/div[1]/main/div[2]/div/article/div[1]/div[3]'
                elif journal == 'agenciabrasil':
                    xpath = '/html/body/main/div[2]/div/div/div[1]/div/div[5]'
                elif journal == 'estadao':
                    xpath = '/html/body/div[1]/main/article'
                elif journal == 'f5folha':
                    xpath = '/html/body/div[9]/div/div[1]/article/div[3]'
                elif journal == 'poder360':
                    xpath = '/html/body/div[1]/main/div[2]/div/div[1]/article/div[1]/div[3]'
                elif journal == 'g1':
                    xpath = '/html/body/div[2]/main/div[4]/article/div[4]'
                elif journal == 'serasa':
                    xpath = '/html/body/div[1]/div/div/main/div/div/div/div/div/div/div[2]/div[1]/div/div[2]/div/div[1]/div[5]'
                elif journal == 'noticiasuol':
                    xpath = '/html/body/div[1]/main/article/div[1]/div[2]/div/div/div'
                elif journal == 'economia':
                    xpath = '/html/body/div[4]/article/div[2]/div/div[1]/div/div[2]/div[1]/div[5]'
                elif journal == 'uol':
                    xpath = '/html/body/div[4]/article/div[2]/div/div[1]/div/div[2]/div[1]/div[4]'
                elif journal == 'em':
                    xpath = '/html/body/div[2]/div[5]/div[1]/div[1]/div/div/div/div[4]/div[1]/div/div[4]/div[2]'
                elif journal == 'unicesumar':
                    xpath = '/html/body/div[1]/div/div[1]'
                elif journal == 'contraponto':
                    xpath = '/html/body/div[1]/div/div/div[2]/div/div[1]/article/div[2]'
                elif journal == 'cm7brasil':
                    xpath = '/html/body/main/div/div[2]/div/article'
                elif journal == 'folhafinanceira':
                    xpath = '/html/body/div[2]/div/main/div/div/div[1]/div[1]/article/div[4]'
                elif journal == 'jornalcontabil':
                    xpath = '/html/body/div[4]/div[2]/div/div/article/div/div/div[1]/div/div[1]/div/div[2]/div/div[3]/div/div[1]/div[1]'
                else:
                    print(f"Jornal não mapeado: {journal}")
                    continue

                text_element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, xpath))
                )
                extracted_text.append(text_element.text)

            except Exception as e:
                print(f"Erro ao carregar texto do site {journal}: {e}")

        driver.quit()
        return extracted_text



In [25]:
automation = GetData()

In [26]:
text = automation.from_website(references_links=links)

Getting data from: https://g1.globo.com/economia/noticia/2023/02/08/golpes-no-carnaval-troca-de-cartao-e-mudanca-de-valor-sao-fraudes-mais-comuns-veja-como-se-proteger.ghtml
Erro ao carregar texto do site g1: Message: 
Stacktrace:
	GetHandleVerifier [0x0x7ff7b9eafe75+79173]
	GetHandleVerifier [0x0x7ff7b9eafed0+79264]
	(No symbol) [0x0x7ff7b9c69e5a]
	(No symbol) [0x0x7ff7b9cc0586]
	(No symbol) [0x0x7ff7b9cc083c]
	(No symbol) [0x0x7ff7b9d14247]
	(No symbol) [0x0x7ff7b9ce89af]
	(No symbol) [0x0x7ff7b9d1100d]
	(No symbol) [0x0x7ff7b9ce8743]
	(No symbol) [0x0x7ff7b9cb14c1]
	(No symbol) [0x0x7ff7b9cb2253]
	GetHandleVerifier [0x0x7ff7ba17a2ad+3004797]
	GetHandleVerifier [0x0x7ff7ba1746fd+2981325]
	GetHandleVerifier [0x0x7ff7ba193350+3107360]
	GetHandleVerifier [0x0x7ff7b9eca9fe+188622]
	GetHandleVerifier [0x0x7ff7b9ed228f+219487]
	GetHandleVerifier [0x0x7ff7b9eb8dc4+115860]
	GetHandleVerifier [0x0x7ff7b9eb8f79+116297]
	GetHandleVerifier [0x0x7ff7b9e9f528+11256]
	BaseThreadInitThunk [0x0x7ff96

In [31]:
df_text = pd.DataFrame(text)

In [33]:

df_text.to_csv(r'..\src\data\processed\bronze\extra_text.csv', sep='|')